<a href="https://colab.research.google.com/github/MJMortensonWarwick/DSML2223/blob/main/1_1_Regression_(Stats_vs_ML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression in Scikit-Learn

### Traditional Statistics Approach
As above, its a Linear Regression tutorial (yay), using the popular (indeed some may say seminal) machine learning library scikit-learn. As per usual, we will start by installing and importing:

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd  
import seaborn as sns 

# Only works on Jupyter/Anaconda
%matplotlib inline  

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

We will also use one of the inbuilt datasets included with _scikit\-learn_:

In [2]:
# import a standard dataset - the Boston house price index
from sklearn.datasets import load_boston
boston_dataset = load_boston()

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

As the comment says, this is a well known dataset ... indeed Andrew Ng's first dataset in his first lecture of the first mainstream MOOC in machine learning. Let's look at it:

In [ ]:
# show the dataset
print(boston_dataset)

# print a return space
print('\n')

# show the keys
print(boston_dataset.keys())

As we can see this is actually a dictionary with 'data' (the X's we can use), 'target' (the Y's), 'feature_names' (the names of all of the columns that represent X), 'DESCR' (a description of the data) and 'filename' (I'll let you guess). With this in mind let's add the X's to a DataFrame:

In [ ]:
# convert to data frame using Pandas
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston.head()

We will also create a variable for the Y value:

In [ ]:
# create a separate Y value
boston_Y = boston_dataset.target
boston_Y

From here we can build our model. Let's recall that the model we want is of the form:

$ Y = a + b_1x_1 + b_2x_2 + [...] + b_13x_13 + e$

Where 
$Y$ is our target variable (boston_Y), $a$ is the intercept, the various $b$ values (1 to 13) represent the effect of the corresponding $x$ values (1 to 13) on the slope and $e$ is the error.

We start by defining a model and 'fitting' it to the data:

In [ ]:
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()

# fit the model to the training data
lin_model_fit = lin_model.fit(boston, boston_Y)

Here we have created a Linear Regression object (lin_model) and then an object based on fitting it to our data. In other words, lin_model_fit is the Linear Regression process applied to our data.

From this object we can establish the remainder of our formula:

In [ ]:
# print the alpha value of the model (intercept)
print("Alpha/intercept (a)")
print(lin_model_fit.intercept_)

print("\n")

# print the beta values of the model (co-efficients)
betas = lin_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients (b1 to b13)")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas[counter], 4)))
    counter +=1

We can also calculate the root mean squared error (RMSE) and the $R^2$ score:

In [ ]:
# predict every Y value in the dataset
boston_predict = lin_model_fit.predict(boston)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(boston_Y, boston_predict)))
r2 = r2_score(boston_Y, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print(f'RMSE is {rmse}')
print(f'R2 score is {r2}')

Overall this is a good model - 
 scores above 0.7 are considered very much publishable/successful when building regression models! RMSE, the square root of the sum of each error term squared, is more domain specific (and often used for module comparison), but in this case means our forecasted property prices are incorrect by, on average, \$4,679. Given that many properties sell for more than $50,000 this is not too bad.

Keep scrolling, we'll next relook at this with a more "machine learning" mindset :)

### Machine Learning Approach
We'll redo our linear regression and this time we'll make it more "machine-learning-y". Actually, arguably, all we need to do is via splitting our data into training and test:

In [ ]:
# split data into training and test
from sklearn.model_selection  import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(boston, boston_Y, test_size = 0.2)

# print the shapes to check everything is OK
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

The "shapes" shown here are number of rows by number of columns ... i.e. the shape of "X_train" is 404 rows by 13 columns. We'd expect the 13 columns (13 X's), and we can quickly do some math on the 506 row numbers ... 
 ... to see that 404 is rougly 80 % of the data (our "test_size" was 20% so our training data should be 80%) and that this all looks correct. The Y "shapes" show only rows but this is because we have only 1x value (1x column).

Using these subsets we can train the Linear Regression model on "X_train" and "Y_train", and then use this model to predict "X_test". If we can compare the predictions on "X_test" with the real values of "Y_test" this gives us a measure of how well our computer has learned to predict house prices:

In [ ]:
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()

# fit the model to the training data
lin_model_fit = lin_model.fit(X_train, Y_train)

# predict the data
boston_predict = lin_model_fit.predict(X_test)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(Y_test, boston_predict)))
r2 = r2_score(Y_test, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

Compared to the previous Notebook our RMSE is lower and 
 higher (i.e. better in both cases)! This is a little suprising but probably just chance on how the data was split (random error means the metrics are superior). However, we can conclude that this hasn't made our model worse.

What it has done, however, is meant we have a model that is tested against its ability to make predictions on unseen data ... i.e. that it has learned about the process of predicting house prices and can do this well.

#### L1 Regularisation (LASSO)
$L1$ Regression is performed in much the same way as Linear Regression. We have one other value (a hyperparameter - more on these later) of $\alpha$ which determines how much influence the 
 penalty has on how the line is fit in the model. We will dodge the issue of what this should be for now and just randomly set it as 0.5

In [ ]:
from sklearn.linear_model import Lasso
l1_model = Lasso(alpha=0.5)

# fit the model to the training data
l1_model_fit = l1_model.fit(X_train, Y_train)

# predict the data
boston_predict = l1_model_fit.predict(X_test)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(Y_test, boston_predict)))
r2 = r2_score(Y_test, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

Performance is slightly lower than for our normal linear model. However... this is somewhat to be expected and actually performance is only slightly worse. This may be a small price to pay for ensuring that our model is a little more robust to changing data.

Where we should see a much more fundamental difference is in the beta-weights/coefficients of the two models (the $b$ values). Let's compare the two:

In [ ]:
# print the beta values of the model (co-efficients)
betas = lin_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - UNREGULARISED")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l1 = l1_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - LASSO")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l1[counter], 4)))
    counter +=1

As we can see some of the beta weights are now 0 ("INDUS", "CHAS" and "NOX" - of which "CHAS" and "NOX" were fairly large weights in the original model). We have effectively removed these features (X's) from the model - they are now (for instance) $0 * INDUS$ which will obviously be zero and therefore won't change the calculation of $Y$. We have performed feature selection - removing X's from the model.

We have also minimised the impact (weight) of all the X's. For instance, "RM" had a beta of 3.77 in the original model and only 0.74 in the LASSO model.

Overall we have made the model less reliant on certain features and removed others entirely (reducing model complexity). Both these elements mean our model is more robust/protected against over-fitting ... and the cost is only 0.05 in terms of $R^2$ score.

#### L2 Regularisation (Ridge)
$L2$ regularisation is performed in a very similar way in scikit-learn ... and again we will set the hyperparameter $\alpha$ to the arbitrary value of 0.5:

In [ ]:
from sklearn.linear_model import Ridge
l2_model = Ridge(alpha=0.75)

# fit the model to the training data
l2_model_fit = l2_model.fit(X_train, Y_train)

# predict the data
boston_predict = l2_model_fit.predict(X_test)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(Y_test, boston_predict)))
r2 = r2_score(Y_test, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

Here we have model performance that slightly improves upon the original model ... while also adding regularisation protection. Let's compare the three beta weights:

In [ ]:
# print the beta values of the model (co-efficients)
betas = lin_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - UNREGULARISED")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l1 = l1_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - LASSO")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l1[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l2 = l2_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - RIDGE")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l2[counter], 4)))
    counter +=1

As with L1/LASSO regression all our beta values are lower than in the original model - and therefore each feature is less influential. However, none have been reduced to zero. L2 cannot perform feature selection (deleting X's) unlike with L1.

So better performance and regularisation added - seems like a good deal right? However, before we call it a day we have one other regularisation method we may try.

#### ElasticNet
ElasticNet uses both $L1$ and $L2$ regularisation. We now have an additonal value to set (hyperparameter - still going to come back to these) the l1_ratio. As the name suggests the l1_ratio determines the proportion that is L1 versus L2 so "1" would be just L1 and "0" would just be L2. We'll arbitrarily go with 0.5 again.

In [ ]:
from sklearn.linear_model import ElasticNet
enet_model = ElasticNet(alpha=0.5, l1_ratio=0.25)

# fit the model to the training data
enet_model_fit = enet_model.fit(X_train, Y_train)

# predict the data
boston_predict = enet_model_fit.predict(X_test)

# calculate RMSE (root mean square error) and R^2 (predictive power)
from sklearn.metrics import mean_squared_error, r2_score
rmse = (np.sqrt(mean_squared_error(Y_test, boston_predict)))
r2 = r2_score(Y_test, boston_predict)

# print the performance metrics
print("Model performance")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

Unsurprisingly the result is somewhere between the results of L1 and L2. Let's also check the weights:

In [ ]:
# print the beta values of the model (co-efficients)
betas = lin_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - UNREGULARISED")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l1 = l1_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - LASSO")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l1[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_l2 = l2_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - RIDGE")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_l2[counter], 4)))
    counter +=1
    
print("\n")

# print the beta values of the model (co-efficients)
betas_enet = enet_model_fit.coef_
counter = 0
for col in boston.columns:
    if counter == 0:
        print("Beta weights/co-efficients - ELASTICNET")
        print("-----------------------------------------")
    print(col + ": " + str(round(betas_enet[counter], 4)))
    counter +=1

Again, as we may expect, the results are somewhere between the two. "CHAS" and "NOX" have betas of zero (have been removed) but not "INDUS". All other beta weights are reduced to somewhere between the L1 and L2 models.

**BONUS! Which is best?**
Hahahahaha ... you know what I'm going to say:

## IT DEPENDS!
If you are not worried about overfitting use the original model (but if you don't care about overfitting are you doing ML?). If you have a lot of features and want some removed you may use L1 or Elastic Net. If you care most about performance use the L2 model. In this case I'd probably go with L2 as there aren't a lot of features and the performance is best.

### Hyperparameter Optimisation
What are the right values for the hyperparameters? In this case we just don't know, but we can try to optimise a solution!

The following hyperparameters should be optimised:
1.   Linear Regression - no hyperparameters
2.   Lasso Regression - $\alpha$ (alpha)
2.   Ridge Regression - $\alpha$ (alpha)
2.   ElasticNet - $\alpha$ (alpha) and $L1$ ratio (how much of the penalty should be $L1$ and how much $L2$)

In [ ]:
from sklearn.linear_model import Lasso

from sklearn.model_selection import GridSearchCV

tuned_parameters = [{'alpha': [0.25, 0.5, 0.75]}] # test the listed alpha values

scores = ['r2'] # test for R^2

for score in scores:
    print("# Tuning hyperparameters for %s" % score)
    print("\n")
    clf = GridSearchCV(Lasso(), tuned_parameters, cv=5,
                       scoring= score)
    clf.fit(X_train, Y_train)
    print("Best parameters set found on the training set:")
    print(clf.best_params_)
    print("\n")

Here we have decide (fairly arbitarily) to test for $R^2$. For comparison sake we could also test for _RMSE_:

In [ ]:
from sklearn.linear_model import Lasso

from sklearn.model_selection import GridSearchCV

tuned_parameters = [{'alpha': [0.25, 0.5, 0.75]}] # test the listed alpha values

scores = ['neg_root_mean_squared_error'] # test for RMSE

for score in scores:
    print("# Tuning hyperparameters for %s" % score)
    print("\n")
    clf = GridSearchCV(Lasso(), tuned_parameters, cv=5,
                       scoring= score)
    clf.fit(X_train, Y_train)
    print("Best parameters set found on the training set:")
    print(clf.best_params_)
    print("\n")

In this case there is the same result. Let's also test Ridge (L2) and ElasticNet:

In [ ]:
from sklearn.linear_model import Ridge

tuned_parameters = [{'alpha': [0.25, 0.5, 0.75]}] # test the listed alpha values

scores = ['r2'] # test for RMSE and R^2

for score in scores:
    print("# Tuning hyperparameters for %s" % score)
    print("\n")
    clf = GridSearchCV(Ridge(), tuned_parameters, cv=5,
                       scoring= score)
    clf.fit(X_train, Y_train)
    print("Best parameters set found on the training set:")
    print(clf.best_params_)
    print("\n")

In [ ]:
from sklearn.linear_model import ElasticNet

tuned_parameters = [{'alpha': [0.25, 0.5, 0.75], # test the listed alpha values
                     'l1_ratio': [0.25, 0.5, 0.75]}] # test the listed L1 ratio values

scores = ['neg_root_mean_squared_error', 'r2'] # test for RMSE and R^2

for score in scores:
    print("# Tuning hyperparameters for %s" % score)
    print("\n")
    clf = GridSearchCV(ElasticNet(), tuned_parameters, cv=5,
                       scoring= score)
    clf.fit(X_train, Y_train)
    print("Best parameters set found on the training set:")
    print(clf.best_params_)
    print("\n")

We can see the "best" hyperparameters based on $R^2$. The final list is:
1.   Linear Regression - still no hyperparameters
2.   Lasso Regression - $\alpha = 0.25$
2.   Ridge Regression - $\alpha = 0.25$
2.   ElasticNet - $\alpha = 0.25$ and $L1$ ratio $= 0.75$

_Note, as we will disucss it is possible you may have different results!_

With this now complete we can update the algorithms and build new models:

In [ ]:
from sklearn.linear_model import LinearRegression

# fit the LR model to the training data
lin_model = LinearRegression()

# fit the Lasso model
l1_model = Lasso(alpha=0.25)

# fit the Ridge model
l2_model = Ridge(alpha=0.25)

# fit the ElasticNet model
enet = ElasticNet(alpha=0.25, l1_ratio=0.75)

# make a list of models to iterate (loop) through
models = [lin_model, l1_model, l2_model, enet]

for model in models:
  # fit model
  fitted_model = model.fit(boston, boston_Y)

  # predict every Y value in the dataset
  boston_predict = fitted_model.predict(boston)

  # calculate RMSE (root mean square error) and R^2 (predictive power)
  from sklearn.metrics import mean_squared_error, r2_score
  rmse = (np.sqrt(mean_squared_error(boston_Y, boston_predict)))
  r2 = r2_score(boston_Y, boston_predict)

  # print the performance metrics
  print(f' {model} - Model performance')
  print("--------------------------------------")
  print(f'RMSE is {rmse}')
  print(f'R2 score is {r2}')
  print('\n')

Linear and Ridge still perform the best (around the same as before), but we can see that Lasso and ElasticNet have considerably improved. If we believe it is important to do "feature selection" (i.e. removing features from the model), we may well now prefer Lasso (L1) even with slightly lower $R^2$